# Best Model, train for 100 epoch, NAdam, larger dataset.


In [1]:
from pathlib import Path
import pandas as pd

from utils import *
from dataloaders import *
from data_preparation import *
from allResNets import *
from VGGs import *
from MobileNetV3 import *
from engine import *
from efficientNet import *

change_to_disk()
data_dir = Path("clean_split_5k/")
models_path = Path('Models/')
saves = Path('Trainning_results_saves/')

In [2]:
data_transform = transforms.Compose([
    # Resize our images to 224x224
    transforms.Resize(size=(224, 224)),
    
    # Rotate the images randomly (just to make it as independent from the position of the pipeline)
    transforms.RandomRotation(180),

    # Random crop
    #transforms.RandomCrop(140),

    #Random horizontal flip
    transforms.RandomHorizontalFlip(p=0.5),

    # Turn the image into a torch.Tensor
    transforms.ToTensor(),

    # Normalize
    transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
])

BATCH_SIZE = 64
train_dl, validation_dl, test_dl, train_data, validation_data, test_data, class_names = create_dataloaders(data_dir, data_transform, batch_size = BATCH_SIZE)

In [3]:
# Things to play with
# Hyperparameters -> NUM_EPOCHS - 100 | Learning_rate - 0.001 
# Functions -> Optimizer - Rprop | Loss Function - Cross Entropy Loss

NUM_EPOCHS = 150
lr = 0.001
device = get_device()

mobnet_sml = MobileNetV3("small")

loss_fn = nn.CrossEntropyLoss()
nadam_optim = torch.optim.NAdam(params=mobnet_sml.parameters(), lr=lr)

In [4]:
mobnet_sml.to(device)
save_name = "mobnet_sml"
train_mobnet_sml_results, train_time_mobnet_sml = train(mobnet_sml, train_dl, validation_dl, optimizer=nadam_optim, loss_fn=loss_fn, epochs=NUM_EPOCHS, name_save=save_name, device=device)

SyntaxError: positional argument follows keyword argument (4277160769.py, line 3)

In [ ]:
eval_mobnet_sml_results = eval_model(mobnet_sml, test_dl, loss_fn = loss_fn, accuracy_fn=accuracy_fn, device = device, dummy_input= torch.rand(32,3,224,224).to(device))

In [ ]:
plot_loss_curves(train_mobnet_sml_results,"MobileNet V3 Small")

In [ ]:
plot_confusion_matrix(mobnet_sml, test_dl, device, class_names, "MobileNetV3 Small")